In [232]:
import numpy as np
import tensorflow.keras
import pickle
from tensorflow.keras.layers import Dense,Conv2D,Activation,Dropout,ReLU,Flatten,MaxPooling2D,BatchNormalization
import cv2
import time
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import os
%matplotlib inline

In [277]:
# In this tutorial we are going to use Microsoft's Cat and Dog Dataset
# You can download it from here

link = "https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765"

# Data preprocessing

In [218]:
# First we have to convert all the images into arrays
# for that we are going to create a function

In [151]:
# location of the data

Location='C:\path\to\your\PetImages'

# in this location i have two folders one is named as Cat and another one is Dog

# First we have to loop through both the folders and conver all the images into arrays 

def prepare_Data(Dir,image_size):
    data=[]                                                  # Creating empty list to store data
    
    categories=os.listdir(Dir)                               # Storing lables(folder names) in a list
    
    for category in categories:                              # Looping through each and every folder
        
        path=os.path.join(Dir,category)                      # joining current folder name with the path
        
        target=categories.index(category)                    # Storing the particular lable as target ['cat','Dog']
        
        for img in os.listdir(path):                         # Looping through the images 
            
            image=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE) # Reading the images
            try:
                image1=cv2.resize(image,(image_size,image_size)) # Images might be in different dimensions so resize it to same dimension
                
                data.append([image1,target])               # Appending resized image and its lable
            except Exception as e:
                pass
    np.random.shuffle(data)                                # Shuffling the data to avoid ["cat","cat","cat","dog","dog","gog"]
    X=[]
    y=[]
    for feature , lable in data:
        X.append(feature)                                  # Spliting X,y 
        y.append(lable)
    return X,y,categories
            

In [263]:
# Let's Call the function to create dataset

X,y,data=image2array(Location,50)

In [266]:
# Now our data is ready but it is a list we have to convert it into array
X=np.array(X)
X.shape

(24946, 50, 50)

In [267]:

# Now Shape of X is 3 dim but we need to convert it into 4 dim (-1,50,50,1) [No_sam,height,width,Channels]
X=np.array(X).reshape(-1,50,50,1)
y=np.array(y)

In [219]:
# Each and every time we can not prepare data so it is safe to save our data somewhere

# for that i am going to use pickle

# Saving the data into pickle

import pickle

data_in=open('cat_dog_X.pickle','wb')
pickle.dump(X,data_in)
data_in.close()
data_y=open('cat_dog_y.pickle','wb')
pickle.dump(y,data_y)
data_y.close()

In [220]:
# Now we can load our data from pickle

# To open data from pickle
x=open('cat_dog_X.pickle','rb')
X=pickle.load(x)
y=open('cat_dog_y.pickle','rb')
y=pickle.load(y)

In [223]:
# Before feeding into model we have to normalize the data to get better performance

# for that we need to first spilt the data

X_train,X_test,y_train,y_test=train_test_split(X,y)

X_train=X_train/255

X_test=X_test/255


In [278]:
# Everuthing is ready now let's build the CNN model

model = Sequential()

# First Convolution layer
model.add(Conv2D(filters=60,kernel_size=3,input_shape=X.shape[1:],strides=1,padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

# Second Convolution layer
model.add(Conv2D(filters=60,kernel_size=3,input_shape=X.shape[1:],strides=1,padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

# Flatten
model.add(Flatten())

# Fully Connected layer

model.add(Dense(80))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compling the model

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [279]:
# Now our Basic CNN model is ready let's train it

In [280]:
model.fit(X_train, y_train, batch_size=32, epochs=1, validation_data=(X_test, y_test))

585/585 [==============================] - 130s 223ms/step - loss: 0.6420 - accuracy: 0.6795 - val_loss: 0.5090 - val_accuracy: 0.7536


In [276]:
# Our Model works well but is not giving good accuracy. for that we have to tune our model

# I am going to use Keras Tuner

# Refer this page to know more about kerastune

kerastune_DOC = "https://keras-team.github.io/keras-tuner/"

In [259]:
# Let's Build a tuner for our model

from kerastuner.tuners import RandomSearch

def build_function(hp):  # hp is the hyperparameter
    
    # first layer will be unique because it consist input_shape so we are defining it outside the loop'
    
    model = Sequential()
    
    model.add(Conv2D(filters=hp.Int('inupt_layer',10,100,5),kernel_size=3,input_shape=X_train.shape[1:],strides=1,
                     padding='same',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size =2))
    
    # Now it's time to decide how many convolution & Dense layer we need to get good performance. 
    
    # for that we are going to use just a for loop
    
    for i in range(hp.Int("conv_layers",1,5)):       # It will randomly pic no of layer from 1 to 5
        
    # Here we need to specify the convolution layer without input size
        
        model.add(Conv2D(filters=hp.Int(f'hidden_conv_neurons{i}_',10,100,5),kernel_size=3,
              strides=1,padding='same',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size =2))
    
    # after conolution layer we need to flatten the data so let's add one Flatten layer
    
    model.add(Flatten())  # note it is outside the loop
    
    # here we have to use one more for loop for Dense layer
    for j in range(hp.Int('Dense_layer',1,10)):              # It will randomly pic no of layer from 1 to 10
       
    # Here we need to specify the Dense layer without input size
        
        model.add(Dense(units=hp.Int(f"Dense_{j}_layer",50,300,20)))
        
        model.add(Dropout(0.5))             # To avaoid overfitting we are adding dropout it is a regularization technique
        
        model.add(Activation('relu'))
    
    # Finally we need to add output layer
    
    model.add(Dense(1))
    model.add(Activation('sigmoid')) # This is a binary classification so we are using sigmoid
    
    # Now we need to compile the model
    
    model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
# Now our Tuner is ready let's see how it works

In [260]:
tuner = RandomSearch(
    build_function,
    objective='val_accuracy',
    max_trials=1,  
    executions_per_trial=1, 
    directory=LOG_DIR)


In [ ]:
# That's it we have completed the enire model now let's tune it

In [262]:
# i am giving only one epoch because i am not using GPU, so it will take toom much time to run.

# feel free to increase the epoch if you have GPU setup
tuner.search(X_train,y_train,verbose=2,
             epochs=1,
             batch_size=64,
             validation_data=(X_test, y_test))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
inupt_layer       |40                |?                 
conv_layers       |3                 |?                 
inupt_0_layer     |70                |?                 
Dense_layer       |9                 |?                 
Dense_0_layer     |290               |?                 

293/293 - 124s - loss: 0.7112 - accuracy: 0.5067 - val_loss: 0.6932 - val_accuracy: 0.5014

Trial 1 Complete [00h 02m 07s]
val_accuracy: 0.5013628602027893

Best val_accuracy So Far: 0.5013628602027893
Total elapsed time: 00h 02m 07s
INFO:tensorflow:Oracle triggered exit


In [275]:
#After tunig the model we can see the best parameters 
# Also we can store it 
print(tuner.results_summary())

Results summary
Results in 1615052476\untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
inupt_layer: 40
conv_layers: 3
inupt_0_layer: 70
Dense_layer: 9
Dense_0_layer: 290
inupt_1_layer: 10
inupt_2_layer: 10
Dense_1_layer: 50
Dense_2_layer: 50
Dense_3_layer: 50
Dense_4_layer: 50
Dense_5_layer: 50
Dense_6_layer: 50
Dense_7_layer: 50
Dense_8_layer: 50
Score: 0.5013628602027893
None


In [ ]:
# After training 20 epoch i got 84% val_accuracy if we spend more time on this model we can get even more accuracy